In [258]:
import pandas as pd
import re 
import nltk
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Embedding,Dropout,Input,Flatten
from keras.models import Sequential
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split




As Test_data does not contian is_sarcastic column , Inorder Verify our predictions I am spliting train_data into train data and validation data sets for predictions and classification report.

In [259]:
train_data1=pd.read_csv('/content/Train-Data- Sarcasm.csv')
train_data,val_data=train_test_split(train_data1,test_size=0.15)
train_data.head()

,Id,article_link,headline,is_sarcastic
4586,19415,https://www.theonion.com/kids-in-bus-accident-...,kids in bus accident mocked by kids in passing...,1
18907,180,https://www.theonion.com/man-failing-to-heed-h...,man failing to heed harsh lessons of past orde...,1
7703,18359,https://www.huffingtonpost.com/entry/over-300-...,over 300 women chime in after l.a. times detai...,0
15338,3031,https://www.huffingtonpost.com/entry/katy-perr...,"after watching katy perry crash a wedding, you...",0
14115,4538,https://www.huffingtonpost.com/entry/sadiq-kha...,london's new mayor: i won't be able to visit t...,0


In [260]:
test_data=pd.read_csv('/content/Test-Data- Sarcasm.csv')
test_data.head()

,Id,article_link,headline
0,18504,https://www.theonion.com/woman-relieved-soulma...,woman relieved soulmate turned out to be in sa...
1,20811,https://www.huffingtonpost.com/entry/end-the-i...,end the international drug war to control the ...
2,20419,https://www.huffingtonpost.com/entry/donald-tr...,who said it: renowned racist george wallace or...
3,20631,https://local.theonion.com/struggling-local-th...,struggling local theater space put out of its ...
4,1921,https://www.theonion.com/graffiti-artist-no-lo...,graffiti artist no longer putting his heart in it


Here we are removing the Unnecessary contents in Headlines. 

In [261]:
def clean_text(text):
    words_length=[]
    cleaned_data=[]
    for sent in tqdm(text):
        sent=re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',str(sent))
        words=[word for word in sent.split()]
      
        words_length.append(len(words))
        cleaned_data.append(' '.join(words))
    return cleaned_data,words_length

In [262]:
train_data['headline'] , train_words_length = clean_text(train_data['headline'])



100%|██████████| 18161/18161 [00:00<00:00, 206056.65it/s]


In [263]:
test_data['headline'] , test_words_length = clean_text(test_data['headline'])

100%|██████████| 5342/5342 [00:00<00:00, 131873.53it/s]


In [264]:
val_data['headline'],val_words_length=clean_text(val_data['headline'])

100%|██████████| 3206/3206 [00:00<00:00, 118448.10it/s]


In [265]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train_data['headline'])


In [266]:
train_headlines=tokenizer.texts_to_sequences(train_data['headline'])
train_headlines_padded=pad_sequences(train_headlines,maxlen=40,padding='pre')

In [267]:
val_headlines=tokenizer.texts_to_sequences(val_data['headline'])
val_headlines_padded=pad_sequences(val_headlines,maxlen=40,padding='pre')

In [268]:
test_headlines=tokenizer.texts_to_sequences(test_data['headline'])
test_headlines_padded=pad_sequences(test_headlines,maxlen=40,padding='pre')

In [269]:
vocab_size = len(tokenizer.word_index)
model5=Sequential()
model5.add(Input(shape=(40,)))
model5.add(Embedding(vocab_size+1,20))
model5.add(Flatten())
model5.add(Dropout(0.4))
model5.add(Dense(8,activation='relu'))
model5.add(Dropout(0.4))
model5.add(Dense(4,activation='relu'))
model5.add(Dense(1,activation='sigmoid'))

In [270]:

model5.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [271]:
model5.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 40, 20)            489900    
_________________________________________________________________
flatten_12 (Flatten)         (None, 800)               0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 800)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 8)                 6408      
_________________________________________________________________
dropout_25 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_37 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_38 (Dense)             (None, 1)               

In [272]:
model5.fit(train_headlines_padded, train_data['is_sarcastic'], epochs=10)

Epoch 1/10
568/568 [==============================] - 6s 9ms/step - loss: 0.6772 - accuracy: 0.5538
Epoch 2/10
568/568 [==============================] - 5s 9ms/step - loss: 0.3785 - accuracy: 0.8425
Epoch 3/10
568/568 [==============================] - 5s 9ms/step - loss: 0.1898 - accuracy: 0.9297
Epoch 4/10
568/568 [==============================] - 5s 9ms/step - loss: 0.1094 - accuracy: 0.9615
Epoch 5/10
568/568 [==============================] - 5s 9ms/step - loss: 0.0647 - accuracy: 0.9741
Epoch 6/10
568/568 [==============================] - 5s 9ms/step - loss: 0.0508 - accuracy: 0.9768
Epoch 7/10
568/568 [==============================] - 5s 9ms/step - loss: 0.0386 - accuracy: 0.9840
Epoch 8/10
568/568 [==============================] - 5s 9ms/step - loss: 0.0347 - accuracy: 0.9855
Epoch 9/10
568/568 [==============================] - 5s 9ms/step - loss: 0.0311 - accuracy: 0.9854
Epoch 10/10
568/568 [==============================] - 5s 9ms/step - loss: 0.0301 - accuracy: 0.9844

In [273]:
model5.evaluate(train_headlines_padded,train_data['is_sarcastic'])
pred=model5.predict_classes(val_headlines_padded)
print(classification_report(val_data['is_sarcastic'],pred))

568/568 [==============================] - 1s 1ms/step - loss: 5.6548e-04 - accuracy: 1.0000
              precision    recall  f1-score   support

           0       0.85      0.90      0.87      1800
           1       0.86      0.80      0.83      1406

    accuracy                           0.85      3206
   macro avg       0.86      0.85      0.85      3206
weighted avg       0.85      0.85      0.85      3206



In [274]:
data=pd.read_csv('Test-Data- Sarcasm.csv')

testing =data["headline"]

data.head(2)

testing_sequences = tokenizer.texts_to_sequences(testing)
padded_testing_sequences= pad_sequences(testing_sequences,padding="pre",maxlen=40)

pred_value=model5.predict_classes(padded_testing_sequences)

idd=list(data['Id'])

dataSet=[]

for i in range(len(pred_value)):
    dataSet.append([idd[i],pred_value[i][0]])







import requests
dataString = ""
for loop in range(len(dataSet)):
    if loop == 0:
        dataString = str(int(dataSet[loop][0]))+','+str(int(dataSet[loop][1]))
    else:
        dataString = dataString+"\n"+str(int(dataSet[loop][0]))+','+str(int(dataSet[loop][1]))

postData = {}
postData['challengeName'] = 'sarcasm'
postData['userID'] = '918535'
postData['challengeType'] = 'binaryclassification'
postData['submissionsData'] = dataString

url = 'https://8n46gxwibi.execute-api.us-east-2.amazonaws.com/default/computeModelScore'
x = requests.post(url,json=postData)

print(x.text)

{"message":"Internal Server Error"}


In [276]:
dataSet[:20]

[[18504, 1],
 [20811, 0],
 [20419, 0],
 [20631, 1],
 [1921, 1],
 [10062, 1],
 [24913, 0],
 [5952, 1],
 [19347, 1],
 [23288, 0],
 [17919, 1],
 [26271, 0],
 [4298, 1],
 [8108, 0],
 [4750, 0],
 [7984, 0],
 [12587, 0],
 [23827, 0],
 [24857, 0],
 [21260, 1]]